# RET Basic Tutorial

## Tutorial 1 - Agents

At the core of RET simulations are agents - the simple 'beings' which perform tasks and through their interactions create the complex systems level outcomes from their relatively simple individual behaviours.

In RET agents are given tasks to perform - such as move, communicate or fire - via an orders system.  When an agent recevies a task their relevant behaviours are called to tell them how to complete it.  Tasks and behaviours are deliberately kept separate so that what an agent is tasked to do is independent of how it does it.  As an example, this approach allows the same move task to be given to be given to a truck, a self-propelled gun and fixed-gun emplacement.  Each of these agents would have a different move behaviour so that same task could result in the truck finding a fast route along roads to the specified destination, the self-propelled gun finding a slower route using a wider vareity of terrains but which provided cover from aerial ISTAR, and the fixed gun emplacement not moving at all.

### Structure of the basic RetAgent

`Mesa` (the underlying framework on which RET is based) defines an `Agent` class and RET extends this to create the `RetAgent` class.  Details of this class can be found in the documentation but for this tutorial an overview is provided.  This overview is about the fundamental `RetAgent` class on which all other RET classes are based, it should be pretty reflective of most child classes and is a good enough start to understanding the structure of any agent in RET.  If you want to define your own agent types then the advanced tutorial provides some details on how to do it.

#### Required Parameters

- model
  - This refers the `RetAgent` back to the underlying `RetModel` object.  All agents need to be 'owned' by a model object in order to be a part of the simulation.
- pos
  - A 2D or 3D coordinate where the agent will start.  Tutorial 2 covers space in more detail but for now it is worth noting that the simulation will not enjoy it if you specify a position outside of the terrain extents and that the units of the  position are defined when you define the space so be careful and match these positions to the units you give later on.
- name
  - The name of the agent - generally meant to be user friendly rather than an ID.  The agent will be given a unique ID when it is created so this name does not have to be unique.
- affiliation
  - Specifies the 'side' the agent is on.  Currently RET recognises FRIENDLY, NEUTRAL, HOSTILE or UNKNOWN affiliations

#### Other Parameters

All the other parameters in a `RetAgent` object will be filled out by default if not provided when instatiating the object.  The majority of the parameters are specifying behaviours with the following behaviour categories are currently defined in RET:

- Move
- Wait
- Sense
  - This requires that you provide a list of sensors available to the agent
- Communicate orders
- Communicate world view
- Fire
- Disable communications
- Deploy countermeasures
  - This requires you to specofy a list of countermeasures available to the agent

### Behaviours

Behaviours define how a specified task should be carried out.  They can cover everything from different ways of moving and shooting to different ways of using sensors.  They can be very rich, and thus enable quite complex actions, or they can be as simple as a blank bit of code which effectively prohibits the agent from carrying out that task.  The behaviours themselves are coded in Python but any agent based on the `RetAgent` class or a child of it will expect behaviours as parameters when it is initialised.  In this way, behaviours become parameters of the agents as you create them and thus the behaviours can be varied as part of an experiment via the batch runner.

For the purposes of this tutorial we are going to minimise the changes we need to make to the underlying RET system and instead rely on the basic framework to rapidly develop a model.  We know we will need two types of move behaviour - one for ground vehicles and one for UAVs - as well as fire behaviours, sensing behaviours and communication behaviours.

#### Move behaviours

The basic RET framework has two developed move behaviours available at present - `GroundBasedMoveBehaviour` and `AircraftMoveBehaviour`.  In both cases the agents are told to move in a straight line from their current location to the goal location without identyfing a lowest cost/fastest/safest route.  The `GroundBasedMoveBehaviour` will drive the agent along the ground matching the contours of the ground as it moves.  The `AircraftMoveBehaviour` will move the agent in bands which are either at a set absolute altitude or a set altitude above the underlying terrain (excluding culture).  These appear suitable for our needs so there is no requirement to modify them for our study.

In [3]:
import typing
import ret
from ret.agents.agent import RetAgent
from ret.sensing.perceivedworld import PerceivedAgent
from ret.behaviours.move import GroundBasedMoveBehaviour, AircraftMoveBehaviour

#### Fire behaviours

The basic RET framework has a simple firing behaviour built in which requires simple data relating to the radius of effect of the ammunition being fired, the probability of kill, some details around rate of fire and a means of resolving which perceived agents should be fired at and then selecting one to target.  The underlying behaviour seems suitable for a basic model as does the selector but the resolver is likely not suitable for our needs.  The default resolver (which prevents firing on UNKOWN or NEUTRAL affiliated agents but gets FRIENDLY agents to target HOSTILE ones and vice versa) needs some modification to ensure that only appropriate targets are provided as potential targets (for instance we want the air defence agents to fire only at the UAVs and nothing else).  The default target selector will randomly select a target from the resolved list of targets which should be fine once the list is reduced to only appropriate targets by the resolver.

We will need a special resolver for the Air defence units to ensure that they only target UAVs but for the purposes of this simple tutorial we will not create any additional resolvers.  The default target resolver simply looks to see if the  target is of an opposing affiliation.  We want to keep this behaviour for our air defence specific resolver but also wish to ensure that only Air agents are included in our list.  RET has a variety of prebuilt filters which includes selecting only Air agents and in addition has a boolean `And` and `Or` operators for combining filters.  As such, all that is required here is to create a new `AffiliationVisitor` class (the `AffiliationVisitor` in RET is used to determine how to make a decision when that decision is based upon the affiliation of decision maker) which combines the filters for afifilation and the agent type.  We then need to extent the `HostileTargetResolver` so it can make use of the new `AffiliationVisitor`.

In [6]:
from ret.sensing.perceivedworld import And, AirAgents, FriendlyAgents, HostileAgents
from ret.behaviours.fire import HostileAgentAffiliationVisitor, HostileTargetResolver


class HostileAirAgentVisitor(HostileAgentAffiliationVisitor):
    """Extends the HostileAgentAffiliationVisitor to filter Air agents of the opposing affiliation"""

    def visit_hostile(self):
        self.enemy = And([FriendlyAgents(), AirAgents()])

    def visit_friendly(self):
        self.enemy = And([HostileAgents(), AirAgents()])


class AirDefenceHostileTargetResolver(HostileTargetResolver):
    """Hostile target resolver specfic to air defence units.  It ensures that only airborne agents at targetted"""

    def run(self, detector: RetAgent, views: list[PerceivedAgent]):
        visitor = HostileAirAgentVisitor()
        detector.affilitation.accept(visitor)
        return visitor.enemy.run(views)

#### Sensing behaviours

Sensing behaviours in RET are very simple as they simply ask every sensor defined for the agent to run its own detection algorithm.  In this way, the means by which a sensor makes an observation is contained within that sensor.  As such, no changes are required to the underlying sense behaviour but we do need to check to see if the correct type of sensors are available.

There are currently 3 basic types of sensor in RET all of which return a list of perceived agents (note, these might not be real agents).  Perceived agents can be observed at different confidences, currently DETECT, RECOGNISE, IDENTIFY or KNOWN.  The 3 types are distance attenuated, where the ability to detect an agent and the confidence with which you make the observation are purely a function of the distance between observer and target; line of sight, which is the same as distance attenuated but where an observation is only made if there is no underlying terrain between the observer and the target; and culture attenuated, which is the same as line of sight but the effective distance between observer and target is increased if the line of sight goes through culture.

For our purposes these sensor types are sufficient and no further modification is required.

#### Communications behaviours

There are currently two basic communications behaviours in RET - one for communicating orders and one for commuinicating an agents world view.  The communicate orders behaviour simply takes a a list of orders and adds them to the recipient agents orders list.  The communicate world view behaviour takes the agents perception and communicates it to the specified recipients.

In both cases the basic behaviour is suitable for our needs.

### Tasks

We now have all the required behaviours defined, but in order for them to be actioned they need to be given to agents.  In RET behaviours are activated when the owning agent is given a task which requires that behaviour to be used.  We will need to ensure we have all the required task types available to us before we begin constructing orders.